In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [3]:
with_mask.shape

(200, 50, 50, 3)

In [4]:
without_mask.shape

(200, 50, 50, 3)

In [5]:
with_mask = with_mask.reshape(200, 50*50*3)
without_mask = without_mask.reshape(200, 50*50*3)

In [6]:
with_mask.shape

(200, 7500)

In [7]:
without_mask.shape

(200, 7500)

In [8]:
# concatenate two arrays
X = np.r_[with_mask, without_mask]

In [9]:
X.shape

(400, 7500)

In [10]:
labels = np.zeros(X.shape[0])

In [11]:
labels[200:] = 1.0

In [12]:
names = {0: "MASK", 1: "NO MASK"}

In [13]:
labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [ ]:
#pip install scikit-learn

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size = 0.33)

In [17]:
y_test.shape

(132,)

In [18]:
# PCA Principal Component Analysis
from sklearn.decomposition import PCA

In [19]:
pca = PCA(n_components = 3)
x_train = pca.fit_transform(x_train)

In [20]:
x_train[0]

array([-3454.31619255,   705.24595673,    -7.0368441 ])

In [21]:
x_train.shape

(268, 3)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size = 0.20)

In [29]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [30]:
#x_test = pca.fit_transform(x_test)
y_pred = svm.predict(x_test)

In [31]:
accuracy_score(y_test, y_pred)

0.9875

In [32]:
# Blue color in BGR
blue = (255, 0, 0)
green = (0, 255, 0)
red = (0, 0, 255)

In [ ]:
haar_data =cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#video capture 0 - from my main camera
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            
            #face shows just face x y coordinate
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1, -1)
            #face = pca.transform(face)
            # 0 or 1 result of pred
            pred = svm.predict(face)
            pred = int(pred)
            if pred == 1:
                cv2.rectangle(img, (x,y), (x+w, y+h), red, 4)
            else:
                cv2.rectangle(img, (x,y), (x+w, y+h), green, 4)
            n = names[pred]
            cv2.putText(img, n, (x,y), font, 1, (244, 250, 250),2)
            #print(n) #shows NO MASK or MASK
        cv2.imshow('result',img)
        #27 - ASCII of Escape
        if cv2.waitKey(2) == 27:
            break
capture.release()
cv2.destroyAllWindows()